* In order to handle “curse of dimensionality” and avoid issues like over-fitting in high dimensional space, methods like Principal Component analysis is used. 
* This method combines highly correlated variables together to form a smaller number of an artificial set of variables which is called “principal components” that account for most variance in the data.

In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
import sklearn as sk
from sklearn.model_selection import train_test_split
eps=np.finfo(float).eps
from binarytree import tree,Node
from sklearn.metrics import classification_report, confusion_matrix ,accuracy_score
from sklearn.tree import DecisionTreeClassifier
from operator import itemgetter
import copy
import collections
from pylab import *
import matplotlib
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA as PCA
import sys
import copy
from sklearn import metrics

**Calculating mean and variance**

In [2]:
def mean(numbers):
    return sum(numbers)/float(len(numbers))

In [3]:
def stdev(numbers):
    avg = mean(numbers)
    varianceNum=0.0
    for x in numbers:
        varianceNum=varianceNum+pow(x-avg,2)
    variance=varianceNum/float(len(numbers)-1)
    return math.sqrt(variance)

**It is important to normalize the predictor as original predictors can be on the different scale and can contribute significantly towards variance**

In [4]:
df=pd.read_csv("intrusion_detection/data.csv")
df
df1=copy.deepcopy(df)
col=list(df)
mean1={}
for i in col:
    if i!="xAttack":
        mean1[i]=mean(df[i])
std1={}
for i in col:
    if i!="xAttack":
        std1[i]=stdev(df[i])
for i in col:
    if i!='xAttack':
        df[i]=(df[i]-mean1[i])/std1[i]

calculate the covariance matrix for the data which would measure how two predictors move together.
calculate Eigen values and Eigen vector of the above matrix. This helps in finding underlying patterns in the data.(eigen vector for direction and eigen value show domination of a atribute)

In [5]:
X = df.iloc[:,0:29].values
y = df.iloc[:,29].values
X_std=X
meanVector = np.mean(X_std, axis=0)
covMat = (X_std - meanVector).T.dot((X_std - meanVector)) / (X_std.shape[0]-1)
eigValues, eigVectors = np.linalg.eig(covMat)
eig_pairs=[]
for i in range(len(eigValues)):
    t=(np.abs(eigValues[i]), eigVectors[:,i])
    eig_pairs.append(t)
total=0
eig_pairs.sort()
eig_pairs.reverse()
error=0
for i in eig_pairs:
    total+=i[0]
j=0;
arr=[]
for i in eig_pairs:
    error+=((i[0]/total)*100)
    arr.append(i[1])
#     print 
    j=j+1;
    if(error>90):
        break;
arr=np.array(arr)
X1=X_std.dot(arr.T)
X1
l=[];
x=len(X1);
for i in range(0,x):
    l.append(i)
Df = pd.DataFrame(data = X1,
                  index=l)

In [6]:
Df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1.008898,-0.272002,0.029599,0.150049,0.259691,0.578092,-0.058920,-0.174519,0.269306,0.034294,-0.066373,0.280442,-0.057627,0.477103
1,-0.495835,1.120904,0.211957,1.340935,-1.399760,2.614645,-1.432528,0.272442,-0.051265,0.011499,-0.163504,0.214390,-0.061687,0.061050
2,-3.802320,-1.248093,0.058765,0.434411,0.394422,-0.108575,0.140224,0.172885,-0.078086,0.006332,0.009409,0.119713,-0.000965,-0.278396
3,2.103182,-1.050183,-0.037603,-0.002242,0.858467,-0.242828,0.029154,0.230058,-0.034821,-0.003268,0.001687,0.053330,-0.178821,0.776296
4,1.891740,-0.881353,-0.156239,-1.170184,0.583162,0.268427,-0.042166,0.312499,-0.093507,-0.006820,0.055598,0.069313,-0.052597,-0.067159
5,-0.931534,5.539636,-0.210924,-0.960439,1.108901,-1.305849,0.555472,-0.134673,-0.036178,0.006247,0.137377,0.083957,0.078635,-0.853413
6,-3.970992,-1.201614,0.046313,0.354667,0.196783,-0.159701,0.166216,0.112153,-0.074875,0.006651,0.000331,0.096870,0.019887,-0.361433
7,-3.771105,-1.230909,0.065579,0.440614,0.297463,-0.036052,0.167639,0.154046,-0.066964,0.011329,-0.003624,0.164325,0.006305,-0.314749
8,-4.083259,-1.194173,-0.010316,-0.017193,-0.232703,-0.307527,0.195776,0.089591,-0.157903,-0.001290,-0.009632,0.001617,0.051939,-0.594729
9,-3.882452,-1.213180,0.061810,0.442993,0.315523,-0.093964,0.157750,0.141845,-0.065774,0.008931,0.002541,0.134192,0.008674,-0.312799


In [7]:
X1 = df1.iloc[:,0:29].values
y1 = df1.iloc[:,29].values
x = StandardScaler().fit_transform(X1)
pca = PCA(n_components=j)
principalComponents = pca.fit_transform(x)
principalDf = pd.DataFrame(data = principalComponents)

In [8]:
principalDf

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,-1.008902,-0.272003,0.029599,-0.150049,-0.259692,-0.578094,-0.058921,-0.174519,0.269307,0.034294,0.066373,-0.280443,0.057627,-0.477105
1,0.495837,1.120908,0.211958,-1.340940,1.399765,-2.614655,-1.432534,0.272444,-0.051265,0.011499,0.163504,-0.214391,0.061687,-0.061050
2,3.802336,-1.248098,0.058765,-0.434412,-0.394424,0.108576,0.140225,0.172886,-0.078086,0.006332,-0.009409,-0.119714,0.000965,0.278397
3,-2.103190,-1.050187,-0.037603,0.002242,-0.858471,0.242829,0.029154,0.230059,-0.034821,-0.003268,-0.001687,-0.053330,0.178822,-0.776299
4,-1.891748,-0.881356,-0.156240,1.170189,-0.583164,-0.268428,-0.042166,0.312500,-0.093507,-0.006820,-0.055598,-0.069313,0.052597,0.067159
5,0.931537,5.539658,-0.210925,0.960443,-1.108905,1.305854,0.555474,-0.134673,-0.036178,0.006247,-0.137377,-0.083957,-0.078636,0.853417
6,3.971007,-1.201618,0.046314,-0.354668,-0.196784,0.159701,0.166217,0.112154,-0.074876,0.006651,-0.000331,-0.096871,-0.019887,0.361434
7,3.771120,-1.230913,0.065579,-0.440615,-0.297464,0.036053,0.167640,0.154047,-0.066965,0.011329,0.003624,-0.164325,-0.006305,0.314750
8,4.083276,-1.194178,-0.010316,0.017193,0.232704,0.307528,0.195777,0.089592,-0.157903,-0.001290,0.009632,-0.001617,-0.051939,0.594731
9,3.882468,-1.213185,0.061811,-0.442994,-0.315524,0.093964,0.157751,0.141846,-0.065774,0.008931,-0.002541,-0.134193,-0.008674,0.312800
